# Places for Half Earth Future
#### This is a new feature that will show precalculated biodiversity and contextual data for selected geometries around the globe (Up to 20 places per country)
#### Following this first approach there might be more narrative involved and a diffeent approach to how to show the data. Right now, the layer will be accessed from the 'Explore places' tab.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy

In [2]:
path = '../../HE_Data/PlacesHE'

In [ ]:
#Columba livia, SliceNumber = 2233
#gadm1= pd.read_csv('/Users/tamarahuete/Documents/HALF_EARTH/gadm1_precalc_no_geometry.csv')
#json.loads(gadm1[gadm1['NAME_1']=='Nyeri']['birds'].values[0])
#{'SliceNumber': 2330.0, 'percentage_presence': 85.8}

In [3]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [4]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [5]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


### 20 places geometries

In [6]:
places= gpd.read_file(f'{path}/Geometries/places20.geojson') # Scott's geometry

In [7]:
places.head()

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Length,Shape_Area,geometry
0,1,AFG,Afghanistan,1,60.222222,7687.504920,1,442334.303716,1.124109e+10,"POLYGON ((7827944.398 4014125.022, 7829351.252..."
1,2,AFG,Afghanistan,2,60.222222,4706.088317,2,354254.657577,7.043484e+09,"POLYGON ((7884416.986 4117164.050, 7885608.258..."
2,3,AFG,Afghanistan,3,52.312500,11768.366965,3,647899.188567,1.746601e+10,"POLYGON ((7661980.648 3995094.171, 7663197.653..."
3,4,AFG,Afghanistan,4,45.500000,205.123344,4,63354.536671,2.958675e+08,"POLYGON ((7551032.457 3954283.587, 7552040.242..."
4,5,AFG,Afghanistan,5,45.500000,205.123344,5,63647.846097,2.979331e+08,"POLYGON ((7523202.584 3994616.263, 7524210.370..."


In [10]:
# places= gpd.read_file('{0}/Geometries/Top_20_places_per_country_20211123.shp'.format(path)) # Scott's geometry
# places.head()

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry
0,AFG,Afghanistan,1,60.222222,7687.504920,"POLYGON ((6784878.655 4081461.837, 6783664.553..."
1,AFG,Afghanistan,2,60.222222,4706.088317,"POLYGON ((6833826.327 4162440.126, 6832851.109..."
2,AFG,Afghanistan,3,52.312500,11768.366965,"POLYGON ((6641029.409 4066345.857, 6640031.675..."
3,AFG,Afghanistan,4,45.500000,205.123344,"POLYGON ((6544864.953 4033762.857, 6544087.481..."
4,AFG,Afghanistan,5,45.500000,205.123344,"POLYGON ((6520743.383 4065965.621, 6519965.911..."


In [11]:
# places['MOL_ID']= places.index+1

#### Format Biodiversity data

In [8]:
### BIOdiversity 

dff = places.copy()
dff['reptiles']=np.nan
dff['amphibians']=np.nan
dff['mammals']=np.nan
dff['birds']=np.nan


### Paths to data generate inside ArcGIS Pro
paths = {'amphibians':'{0}/Csvs/Sample_amph_nozeroes.csv'.format(path),
         'birds':'{0}/Csvs/Sample_birds_nozeroes.csv'.format(path),
         'mammals':'{0}/Csvs/Sample_mammals_nozeroes.csv'.format(path),
         'reptiles':'{0}/Csvs/Sample_reptiles_nozeroes.csv'.format(path)}

### Ids of lookup tables for each taxa in ArcGIS online
lookups = {'amphibians':'a641a4cd269345dea93b8bcb1cb66676',
         'birds':'4d8698734b654bb9bb7a61d9af314c76',
         'mammals':'84d3c71caf97479d85f620a4ee217d68',
         'reptiles':'b720e84869624e5482b25338b27a54f2'}

In [9]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [10]:
%%time
### Loop over each set of data and add all results to one dictionary that will be a text field in the Feature Service
for taxa in paths.keys():
    print(taxa) 
    df = pd.read_csv(paths[taxa])
        
    ### Get species area against global species range:
    lookup = getHTfromId(lookups[taxa])
    df = df.merge(lookup[['SliceNumber','range_area_km2']], how='left',on = 'SliceNumber')
    df = df.merge(places[['MOL_ID','AREA_KM2']])
    
    if taxa == 'mammals':
        df['per_global'] = round(df['presence']/df['range_area_km2']*100,2)
        df['per_aoi'] = round(df['presence']/df['AREA_KM2']*100,2)
    else:
        df['per_global'] = round(df[taxa]/df['range_area_km2']*100,2)
        df['per_aoi'] = round(df[taxa]/df['AREA_KM2']*100,2)
    
    ### Format as array of dicts:
    for i in dff.MOL_ID:
        species = []
        for j in df[(df.MOL_ID== i)].SliceNumber.unique():
            sp_dict = {'SliceNumber':int(df[(df.MOL_ID== i) & (df.SliceNumber==j)].SliceNumber.values[0]),
                       'per_global':df[(df.MOL_ID== i) & (df.SliceNumber==j)].per_global.values[0],
                       'per_aoi':df[(df.MOL_ID== i) & (df.SliceNumber==j)].per_aoi.values[0]
                      }
            species.append(sp_dict)
        dff.loc[dff['MOL_ID']==i,taxa] =json.dumps(species)
    print(f'finished MOL_ID ={i}')
pd.set_option('display.max_colwidth',None)
test= pd.DataFrame(dff[dff.columns[~dff.columns.isin(['geometry'])]])
dff.to_file("../../HE_Data/PlacesHE/top_places_precalc_biodiv.geojson",driver='GeoJSON')
pd.reset_option('display.max_colwidth')

#### Don't export as shapefile because it truncates the string length

amphibians
finished MOL_ID =1925
birds
finished MOL_ID =1925
mammals
finished MOL_ID =1925
reptiles
finished MOL_ID =1925
CPU times: user 34min 28s, sys: 25.5 s, total: 34min 53s
Wall time: 35min 51s


In [11]:
dff.head(2)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Length,Shape_Area,geometry,reptiles,amphibians,mammals,birds
0,1,AFG,Afghanistan,1,60.222222,7687.504920,1,442334.303716,1.124109e+10,"POLYGON ((7827944.398 4014125.022, 7829351.252...","[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_..."
1,2,AFG,Afghanistan,2,60.222222,4706.088317,2,354254.657577,7.043484e+09,"POLYGON ((7884416.986 4117164.050, 7885608.258...","[{""SliceNumber"": 9, ""per_global"": 0.2, ""per_ao...","[{""SliceNumber"": 951, ""per_global"": 4.19, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.12, ""per...","[{""SliceNumber"": 92, ""per_global"": 0.06, ""per_..."


In [12]:
dff.to_file("{0}/top_places_precalculated_biodiv.geojson".format(path),driver='GeoJSON') 
dff.to_csv("{0}/top_places_precalculated_biodiv.csv".format(path))

In [13]:
dff.loc[dff['MOL_ID']==20,'mammals'].values[0]

'[{"SliceNumber": 575, "per_global": 1.62, "per_aoi": 87.54}, {"SliceNumber": 605, "per_global": 0.0, "per_aoi": 77.68}, {"SliceNumber": 607, "per_global": 0.01, "per_aoi": 86.61}, {"SliceNumber": 609, "per_global": 0.0, "per_aoi": 94.73}, {"SliceNumber": 615, "per_global": 0.04, "per_aoi": 67.36}, {"SliceNumber": 635, "per_global": 0.0, "per_aoi": 90.78}, {"SliceNumber": 1176, "per_global": 0.09, "per_aoi": 44.75}, {"SliceNumber": 1613, "per_global": 0.05, "per_aoi": 11.83}, {"SliceNumber": 1784, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 1787, "per_global": 0.0, "per_aoi": 95.77}, {"SliceNumber": 1955, "per_global": 0.01, "per_aoi": 65.97}, {"SliceNumber": 2106, "per_global": 0.01, "per_aoi": 94.73}, {"SliceNumber": 2240, "per_global": 0.01, "per_aoi": 81.62}, {"SliceNumber": 2335, "per_global": 0.01, "per_aoi": 42.9}, {"SliceNumber": 2341, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 2811, "per_global": 0.17, "per_aoi": 69.22}, {"SliceNumber": 2889, "per_global": 0

### Format contextual data 

In [36]:
### Read files (Files coming from ArcGIS project Places_HE_Future and expoprted as csv)
### Top 20 places original geometries from Scott: https://eowilson.maps.arcgis.com/home/item.html?id=4848c6b08fac4fa5bff40e9331b6d291
elu= pd.read_csv('{0}/Csvs/Places_elu.csv'.format(path))
le= pd.read_csv('{0}/Csvs/Places_encroachment.csv'.format(path))
pop= pd.read_csv('{0}/Csvs/Places_pop.csv'.format(path))
wl= pd.read_csv('{0}/Csvs/Places_wdpa_list.csv'.format(path))
wp= pd.read_csv('{0}/Csvs/Places_wdpa_percentage.csv'.format(path))

cont_lookups={'elu':'83802a7fa3d34c1fa40844fc14683966',
              'landencroach':'b0ad58e2b2404eec811b711b604b9d55'}

In [43]:
elu = elu.rename(columns ={'OBJECTID_1':'MOL_ID'})
le = le.rename(columns ={'OBJECTID_1':'MOL_ID'})
pop = pop.rename(columns ={'OBJECTID_1':'MOL_ID'})

For elu layer there are some rows that were not processed in Arcgis Pro (the geometries were not rasterized). To know which geometries are not included in elu table do this and repeat analysis in arcgis pro only for those rows.

In [16]:
l = list(range(1,1926))

In [17]:
l = list(set(dff.MOL_ID) - set(elu.MOL_ID))
len(l) # number of geometries that have no elu data, repeat analysis in ArcGIS Pro for those

95

In [18]:
# Read elu table with new rows
elu2= pd.read_csv('{0}/Csvs/Places_elu_nulls.csv'.format(path))

In [19]:
# Merge elu datasets
elu = elu.append(elu2)
elu.head(1)

,OID_,MOL_ID,COUNT,AREA,MAJORITY
0,1,1,148821.0,0.750596,184


In [20]:
l = list(set(dff.MOL_ID) - set(elu.MOL_ID))
len(l) # those geometries don't have elu data

28

In [75]:
### Column names needed: 
# 'percent_irrigated', 'percent_rainfed',
#        'percent_rangeland', 'percent_urban', 'population_sum',
#        'majority_land_cover_climate_regime', 'land_cover_majority',
#        'climate_regime_majority',

Index(['Unnamed: 0', 'GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'MOL_ID',
       'AREA_KM2', 'reptiles', 'amphibians', 'mammals', 'birds',
       'percentage_protected', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'population_sum',
       'majority_land_cover_climate_regime', 'land_cover_majority',
       'climate_regime_majority', 'country_size'],
      dtype='object')

In [21]:
## Add contextual data: ELU
dff = dff.merge(elu[['MOL_ID','MAJORITY']], how='left', on = 'MOL_ID').rename(columns={'MAJORITY':'majority_land_cover_climate_regime'})

In [22]:
dff.head(1)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Length,Shape_Area,geometry,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime
0,1,AFG,Afghanistan,1,60.222222,7687.50492,1,442334.303716,1.124109e+10,"POLYGON ((7827944.398 4014125.022, 7829351.252...","[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0


In [23]:
# Retrieve elu lookup table to see to see the correspondences for that elu code
elu_lookup = getHTfromId(cont_lookups['elu'])
elu_lookup.head()

,elu_code,elu,lc_type,lf_type,cr_type,ObjectId
0,301,Sub Tropical Moist Forest on Plains,Forest,Plains,Sub Tropical Moist,1
1,201,Warm Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Warm Temperate Dry,2
2,151,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Cool Temperate Dry,3
3,302,Sub Tropical Moist Cropland on Tablelands,Cropland,Tablelands,Sub Tropical Moist,4
4,152,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Tablelands,Cool Temperate Dry,5


In [24]:
# Merge in dataset the required info from lookup table
dff = dff.merge(elu_lookup[['elu_code','lc_type','cr_type']], how='left', left_on = 'majority_land_cover_climate_regime', right_on = 'elu_code')\
    .drop(columns=['elu_code'])\
    .rename(columns={'lc_type':'land_cover_majority','cr_type':'climate_regime_majority'})

In [30]:
dff.head(1)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Length,Shape_Area,geometry,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority
0,1,AFG,Afghanistan,1,60.222222,7687.50492,1,442334.303716,1.124109e+10,"POLYGON ((7827944.398 4014125.022, 7829351.252...","[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry


In [44]:
# Each slicenumber with a type of land encroachment
le.head(1)

,OID_,MOL_ID,COUNT,AREA,Variable,Dimensions,SliceNumber,MIN,MAX,RANGE,MEAN,STD,SUM,MEDIAN,PCT90,percentage_land_encroachment
0,1,1,2846.0,0.197635,Variable_new,SliceNumber,1.0,0.276261,0.814483,0.538222,0.462732,0.133903,1316.934454,0.408784,0.684398,26.31


In [32]:
# Check geometries that have no land encroachment data
l = list(set(dff.MOL_ID) - set(le.MOL_ID))
len(l) # repeat analysis for those

394

In [45]:
# Upload results from second analysis
le2 = pd.read_csv(f'{path}/Csvs/Places_encroachment_nulls.csv')
le2.head(1)

,OID_,MOL_ID,COUNT,AREA,Variable,Dimensions,SliceNumber,MIN,MAX,RANGE,MEAN,STD,SUM,MEDIAN,PCT90,Total,percentage_land_encroachment
0,1,157,250.0,0.017361,Variable_new,SliceNumber,1.0,0.213953,0.86497,0.651016,0.658066,0.121977,164.516578,0.695782,0.750065,1033.0,24.2


In [47]:
le2 = le2.drop(columns={'Total'})
le2.head()

,OID_,MOL_ID,COUNT,AREA,Variable,Dimensions,SliceNumber,MIN,MAX,RANGE,MEAN,STD,SUM,MEDIAN,PCT90,percentage_land_encroachment
0,1,157,250.0,0.017361,Variable_new,SliceNumber,1.0,0.213953,0.864970,0.651016,0.658066,0.121977,164.516578,0.695782,0.750065,24.20
1,2,162,20.0,0.001389,Variable_new,SliceNumber,1.0,0.261984,0.613937,0.351953,0.409149,0.138521,8.182981,0.391272,0.612955,0.93
2,3,223,97.0,0.006736,Variable_new,SliceNumber,1.0,0.190901,0.710594,0.519693,0.415030,0.154798,40.257863,0.380945,0.607992,4.94
3,4,343,1414.0,0.098193,Variable_new,SliceNumber,1.0,0.231417,0.844146,0.612730,0.426742,0.120487,603.412633,0.397267,0.589707,8.95
4,5,344,31.0,0.002153,Variable_new,SliceNumber,1.0,0.357714,0.548353,0.190639,0.431293,0.056941,13.370077,0.421643,0.513450,0.19


In [49]:
# Merge both land encroachment datasets
le = le.append(le2)
le.head(1)

,OID_,MOL_ID,COUNT,AREA,Variable,Dimensions,SliceNumber,MIN,MAX,RANGE,MEAN,STD,SUM,MEDIAN,PCT90,percentage_land_encroachment
0,1,1,2846.0,0.197635,Variable_new,SliceNumber,1.0,0.276261,0.814483,0.538222,0.462732,0.133903,1316.934454,0.408784,0.684398,26.31


In [50]:
# Check geometries that have no land encroachment data
l = list(set(dff.MOL_ID) - set(le.MOL_ID))
len(l) # those geometries don't have le data

193

In [51]:
# See all land encroachment data for each mol_id
long =le[['SliceNumber','MOL_ID','percentage_land_encroachment']].pivot(index='MOL_ID', columns='SliceNumber', values='percentage_land_encroachment').reset_index().rename(columns=
                                                                                                                              {1.0:'percent_irrigated',
                                                                                                                              2.0:'percent_rainfed',
                                                                                                                              3.0:'percent_rangeland',
                                                                                                                                 4.0:'percent_urban'})

In [52]:
long.head(1)

SliceNumber,MOL_ID,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,1,26.31,57.02,15.5,NaN


In [53]:
# How many encroachment values are higher than 100? This can be due to the way the area is calculated. Anyway, the website will show only the le that is 
# majoritary and the % will stop at 100%
long['sum']=long.iloc[:, 1:5].sum(axis=1)
long[long['sum']>100]

SliceNumber,MOL_ID,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,sum
2,3,33.07,6.93,111.57,2.66,154.23
5,7,14.04,0.93,101.45,0.14,116.56
19,22,NaN,23.95,108.24,0.74,132.93
20,23,0.67,147.48,43.64,2.01,193.80
24,28,11.90,118.87,26.94,2.91,160.62
...,...,...,...,...,...,...
1637,1825,4.79,12.90,153.44,8.54,179.67
1685,1878,1.22,50.35,106.65,10.75,168.97
1686,1879,20.71,16.55,81.57,3.66,122.49
1702,1895,NaN,NaN,139.54,NaN,139.54


In [54]:
## Add contextual data: Land Encroachment
dff = dff.merge(long[['MOL_ID','percent_irrigated', 'percent_rainfed', 'percent_rangeland','percent_urban']],on ='MOL_ID',how='left')

In [55]:
dff.head(1)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Length,Shape_Area,geometry,...,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,1,AFG,Afghanistan,1,60.222222,7687.50492,1,442334.303716,1.124109e+10,"POLYGON ((7827944.398 4014125.022, 7829351.252...",...,"[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN


In [56]:
# Population table, we need the SUM attribute
pop.head(1)

,OID_,MOL_ID,COUNT,AREA,SUM
0,1,1,10824.0,0.751667,2.092001e+06


In [57]:
## Add contextual data: POP
dff = dff.merge(pop[['MOL_ID','SUM']],on ='MOL_ID',how='left')
dff.head(1)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Length,Shape_Area,geometry,...,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,SUM
0,1,AFG,Afghanistan,1,60.222222,7687.50492,1,442334.303716,1.124109e+10,"POLYGON ((7827944.398 4014125.022, 7829351.252...",...,"[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN,2.092001e+06


In [58]:
dff = dff.rename(columns ={'SUM':'population_sum'})
dff.head(1)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Length,Shape_Area,geometry,...,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,1,AFG,Afghanistan,1,60.222222,7687.50492,1,442334.303716,1.124109e+10,"POLYGON ((7827944.398 4014125.022, 7829351.252...",...,"[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN,2.092001e+06


In [59]:
# Remove geometries that were removed from calculations in ArcGIS Pro because they gave errors when running the sample tool against the crfs
dff2 = dff.loc[(dff["MOL_ID"]!=62)] # ATL
dff2 = dff2.loc[(dff["MOL_ID"]!=69)] # ATL
dff2 = dff2.loc[(dff["MOL_ID"]!=73)] # ATL
dff2 = dff2.loc[(dff["MOL_ID"]!=194)] # BRZ
len(dff2)

1921

In [60]:
dff2.head(1)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Length,Shape_Area,geometry,...,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,1,AFG,Afghanistan,1,60.222222,7687.50492,1,442334.303716,1.124109e+10,"POLYGON ((7827944.398 4014125.022, 7829351.252...",...,"[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN,2.092001e+06


In [61]:
dff2.loc[dff['MOL_ID']==20,'mammals'].values[0] # check the string to see it's formatted correctly 

'[{"SliceNumber": 575, "per_global": 1.62, "per_aoi": 87.54}, {"SliceNumber": 605, "per_global": 0.0, "per_aoi": 77.68}, {"SliceNumber": 607, "per_global": 0.01, "per_aoi": 86.61}, {"SliceNumber": 609, "per_global": 0.0, "per_aoi": 94.73}, {"SliceNumber": 615, "per_global": 0.04, "per_aoi": 67.36}, {"SliceNumber": 635, "per_global": 0.0, "per_aoi": 90.78}, {"SliceNumber": 1176, "per_global": 0.09, "per_aoi": 44.75}, {"SliceNumber": 1613, "per_global": 0.05, "per_aoi": 11.83}, {"SliceNumber": 1784, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 1787, "per_global": 0.0, "per_aoi": 95.77}, {"SliceNumber": 1955, "per_global": 0.01, "per_aoi": 65.97}, {"SliceNumber": 2106, "per_global": 0.01, "per_aoi": 94.73}, {"SliceNumber": 2240, "per_global": 0.01, "per_aoi": 81.62}, {"SliceNumber": 2335, "per_global": 0.01, "per_aoi": 42.9}, {"SliceNumber": 2341, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 2811, "per_global": 0.17, "per_aoi": 69.22}, {"SliceNumber": 2889, "per_global": 0

In [62]:
dff2.to_file("{0}/top_places_precalculated.geojson".format(path),driver='GeoJSON')
# dff2.to_csv("{0}/top_places_precalculated_all.csv".format(path))
# dff2.to_file("{0}/top_places_precalculated.gpkp".format(path), driver="GPKG")